# Socratic Teacher

<img src="./socratic.png" alt="Alt text" style="width:150px; float:left; margin-right:10px;"/>

This is a program that interacts with chatGPT to create a Socratic dialog about any topic.  
It will gradually build from initial intuitions from a basic to more advanced level.

In [ ]:
import openai
import os

def read_api_key(file_path):
    """
    Reads the API key from a specified file.

    Parameters:
    file_path (str): The path to the file containing the API key.

    Returns:
    str: The API key.
    """
    with open(file_path, 'r') as file:
        return file.read().strip()

def get_initial_question(topic):
    """
    Generate the initial question to start the Socratic method teaching.

    Parameters:
    topic (str): The topic to be taught.

    Returns:
    str: The initial question.
    """
    return f"Let's begin with an intuition about {topic}. What do you already know about this topic?"

def get_next_question(topic, answers, client, complexity_level):
    """
    Generate the next question based on the previous answers and the complexity level.

    Parameters:
    topic (str): The topic to be taught.
    answers (str): The previous answers.
    client: The OpenAI client.
    complexity_level (str): The complexity level of the question.

    Returns:
    str: The next Socratic question.
    """
    prompt = f"The topic is {topic}. Based on the previous answers and at a {complexity_level} level, generate the next question to guide the user to learn more about the topic:\n{answers}"
    return call_chatgpt(prompt, client)

def generate_quiz(topic, answers, client, complexity_level):
    """
    Generate a quiz question to reinforce learning at the specified complexity level.

    Parameters:
    topic (str): The topic to be taught.
    answers (str): The previous answers.
    client: The OpenAI client.
    complexity_level (str): The complexity level of the quiz question.

    Returns:
    str: A quiz question.
    """
    prompt = f"The topic is {topic}. Based on the previous answers and at a {complexity_level} level, generate a quiz question to reinforce learning:\n{answers}"
    return call_chatgpt(prompt, client)


def call_chatgpt(prompt, client):
    """
    Calls the OpenAI API to generate a response based on the provided prompt.

    Parameters:
    prompt (str): The prompt describing the desired response.
    client (openai.OpenAI): The OpenAI client.

    Returns:
    str: The generated response.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.dict()['choices'][0]['message']['content'].strip()


def construct_rich_prompt(topic, client):
    """
    Constructs a richer prompt based on the original topic using ChatGPT.

    Parameters:
    topic (str): The topic to be taught.
    client (openai.OpenAI): The OpenAI client.

    Returns:
    str: The constructed rich prompt.
    """
    prompt = f"Rephrase the following topic in a richer and more detailed way: {topic}"
    return call_chatgpt(prompt, client)

def determine_complexity_level(iteration, total_iterations, start_level, end_level):
    """
    Determine the complexity level based on the iteration, total iterations, start level, and end level.

    Parameters:
    iteration (int): The current iteration number.
    total_iterations (int): The total number of iterations.
    start_level (str): The starting complexity level.
    end_level (str): The ending complexity level.

    Returns:
    str: The complexity level.
    """
    if iteration < total_iterations * 0.3:
        return start_level
    elif iteration < total_iterations * 0.7:
        return "college"
    else:
        return end_level

def main():
    # Load the OpenAI API key
    api_key = read_api_key('./api_key.txt')
    openai.api_key = api_key
    client = openai

    # Get the topic to teach
    topic = input("Please enter the topic to teach: ")
    
    # Construct and confirm the rich prompt
    rich_prompt = construct_rich_prompt(topic, client)
    print(f"Generated Rich Prompt:\n{rich_prompt}\n")
    confirmation = input("Is this what you meant? (yes/no): ")
    if confirmation.lower() != 'yes':
        print("Please restart the program and enter a more specific topic.")
        return

    # Get the number of iterations and complexity levels
    iterations = int(input("How many teaching iterations would you like? (default is 50): ") or 50)
    start_level = input("Enter the starting complexity level (default is 'high-school'): ") or "high-school"
    end_level = input("Enter the ending complexity level (default is 'graduate'): ") or "graduate"
    
    # Initialize the dialogue
    answers = ""
    question = get_initial_question(topic)
    
    for i in range(iterations):
        print(f"\nIteration {i + 1}:")
        print("Question:", question)
        answer = input("Your answer: ")
        answers += f"Q: {question}\nA: {answer}\n"
        
        # Determine the complexity level
        complexity_level = determine_complexity_level(i, iterations, start_level, end_level)
        
        # Generate the next question
        question = get_next_question(topic, answers, client, complexity_level)
        
        # Optionally, generate a quiz
        if i % 10 == 0:  # Generate a quiz every tenth iteration
            quiz = generate_quiz(topic, answers, client, complexity_level)
            print("Quiz:", quiz)
            quiz_answer = input("Your quiz answer: ")
            answers += f"Q: {quiz}\nA: {quiz_answer}\n"
    
    print("\nTeaching session complete.")
    print("Summary of Q&A:")
    print(answers)

if __name__ == "__main__":
    main()
